# Evolutionary Grid Cells

##THE INDUCTION CITIES, Makoto Sei Watanabe, 1994

![alt text](http://www.archilab.org/public/1999/artistes/images/wata11g.jpg)

https://makoto-architect.com/idc2000/index1.html



## An Evolutionary Architecture, John Frazer, 1995

![alt text](http://www.johnfrazer.com/images/evolarch320.jpg)
[download the book](http://www.aaschool.ac.uk/PUBLIC/AAPUBLICATIONS/outofprint.php?item=441)


## FUNCTION MIXER, MVRDV, 2001


![alt text](http://www.creeaza.com/files/arhitectura/4_poze/image082.jpg)
![alt text](http://www.creeaza.com/files/arhitectura/4_poze/image084.jpg)



# Built Examples Nesting / Dense Packing Strategy

## Gifu Kitagana Apartment Building, SANAA
![alt text](http://www.japan-photo.de/D-GIFU-KITAGATA-APARTMENTS60.jpg)

## Kanazawa Museum, SANAA
![alt text](http://archdarwin.com/wp-content/uploads/2018/03/SANAA-kanazawa-meuseum-plan.jpg)


## [Double House Utrecht, MVRDV](https://www.mvrdv.nl/projects/164/double-house-utrecht) 
![alt text](http://api.mvrdv.boerdamdns.nl/media/uploads/project/164/1.jpg?width=1920)

##Children Centre, Sou Fujimoto

![alt text](https://www.architectural-review.com/pictures/1180xany/5/6/3/3043563_soufujimoto_hokkaidochildrencentre.jpg)

# Code Example

## Import

In [0]:
!rm -rf mola
!git clone https://github.com/dbt-ethz/mola.git

Cloning into 'mola'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 1306 (delta 9), reused 11 (delta 3), pack-reused 1281
Receiving objects: 100% (1306/1306), 261.10 KiB | 10.88 MiB/s, done.
Resolving deltas: 100% (835/835), done.


In [0]:
from IPython.display import HTML
import random
from mola.core import *
from mola.grid import Grid
import mola.renderP5JS as renderer2D

##Polyomino Definition

In [0]:
class Polyomino:
    def __init__(self,vertices,color=(1,1,1,1)):
        self.vertices=vertices
        self.color=color
        self.id=0
        self.position=Vertex(0,0,0)
        self.normalise()

    def normalise(self):
        self.bounds=Box()
        for v in self.vertices:
            self.bounds.addPoint(v.x,v.y,v.z)
        for v in self.vertices:
            v.x=v.x-self.bounds.x1
            v.y=v.y-self.bounds.y1
            v.z=v.z-self.bounds.z1
        self.bounds=Box()
        for v in self.vertices:
            self.bounds.addPoint(v.x,v.y,v.z)

##Init

In [0]:
def initPolyminos():
    global polyominos
    polyominos=[]
    for i in range(10):
        vertices=[]
        for j in range(8):
            x=random.randint(0,3)
            y=random.randint(0,3)
            v=Vertex(x,y)
            if v not in vertices:
                vertices.append(v)
        ccolor=(random.uniform(0,1),random.uniform(0,1),random.uniform(0,1),1)
        for n in range(40):
            polymino=Polyomino(vertices)
            polymino.color=ccolor
            polyominos.append(polymino)

def initEnvironment():
    global grid,fitness
    grid=Grid(50,40)
    grid.values=[0]*grid.nX*grid.nY

##Optimiziation

In [0]:
def findBestPosition(polyomino):
    #polymino searches for the lowest position in the grid where it could be placed without conflict
    global grid
    for z in range(grid.nZ):
        for y in range(grid.nY):
            for x in range(grid.nX):
                fits=True
                for v in polyomino.vertices:
                    cX=x+v.x
                    cY=y+v.y
                    cZ=z+v.z
                    if cX<grid.nX and cY<grid.nY and cZ<grid.nZ:
                        if grid.get(cX,cY,cZ)!=0:
                            fits=False
                            break
                    else: 
                        fits=False
                        break
                if fits:
                    for v in polyomino.vertices:
                        cX=x+v.x
                        cY=y+v.y
                        cZ=z+v.z
                        grid.set(polyomino,cX,cY,cZ)
                        polyomino.position=Vertex(cX,cY,cZ)
                    return True
    return False

def test():
    #calculate the overall height or the density
    grid.values=[0]*grid.nX*grid.nY*grid.nZ
    for p in polyominos:
        findBestPosition(p)
    centerOfMass=Vertex()
    nNodes=0
    for z in range(grid.nZ):
        for y in range(grid.nY):
            for x in range(grid.nX):
                if grid.get(x,y,z)!=0:
                    nNodes+=1
    if nNodes==0:return 100000
    return grid.nX*grid.nY*grid.nZ-nNodes

def backup():
    #save the sequence of polyminos
    global backUpList,polyominos
    backUpList=list(polyominos)

def restore():
    # restore the old sequence
    global backUpList,polyominos
    polyominos=list(backUpList)
    test()

def mutate(mutationRate=1):
    # mutate the sequence
    global polyominos
    for i in range(mutationRate):
        rnd1=random.randint(0,len(polyominos)-1)
        rnd2=random.randint(0,len(polyominos)-1)
        temp=polyominos[rnd1]
        polyominos[rnd1]=polyominos[rnd2]
        polyominos[rnd2]=temp

def mutateForBetter(mutationRate=1):
    global fitness
    backup()
    mutate(mutationRate)
    newFitness=test()
    if newFitness>fitness:
        restore()
    else:
        fitness=newFitness

##Run

In [0]:
initPolyminos()
initEnvironment()
fitness=test()

In [0]:
for i in range(50):
  mutateForBetter(3)
test()

559

##Display

In [0]:
renderer2D.beginDraw(500,400)
renderer2D.scale(10,10)
renderer2D.noStroke()
renderer2D.strokeWeight(0.1)
for x in range(grid.nX):
    for y in range(grid.nY):
        if grid.get(x,y)!=0:
            p=grid.get(x,y)
            renderer2D.fill(p.color[0]*255,p.color[1]*255,p.color[2]*255)
            renderer2D.rect(x,y,x+1,y+1)
renderer2D.stroke(0,0,0)
renderer2D.noFill()
renderer2D.rect(0,0,grid.nX,grid.nY)
for x in range(grid.nX):
    for y in range(grid.nY):
        v1=grid.get(x,y)
        if x<grid.nX-1:
          v2=grid.get(x+1,y)
          if v1!=v2:
             renderer2D.line(x+1,y,x+1,y+1)
        if y<grid.nY-1:
          v3=grid.get(x,y+1)
          if v1!=v3:
            renderer2D.line(x,y+1,x+1,y+1)
HTML(renderer2D.endDraw())